In [7]:
import pandas as pd
import numpy as np
 
import tensorflow as tf
 
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [8]:
url= 'https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv'
df = pd.read_csv(url)

In [9]:
df

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...,...
53935,53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,53939,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [10]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

34521 train examples
8631 validation examples
10788 test examples


In [12]:
# Cara untuk membuat dataset tf.data dari pandas dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = df.copy()
  df['target'] = np.where(df['price'] <= 1000, 0, 1)
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds
batch_size = 10 #bath ukuran kecil untuk demonstrasi
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [13]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [14]:
#numeric column
carat = feature_column.numeric_column('carat')
demo(carat)

[[1.01]
 [0.8 ]
 [0.61]
 [0.51]
 [0.53]
 [0.32]
 [0.53]
 [1.55]
 [0.7 ]
 [0.41]]


In [15]:
#bucketized column
carat = feature_column.numeric_column('carat')
carat_buckets = feature_column.bucketized_column(carat, boundaries=[1, 2])
demo(carat_buckets)

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [16]:
#categorical
color_type = feature_column.categorical_column_with_vocabulary_list(
      'color', ['E', 'I','J','D','H', 'G','F'])
 
color_type_one_hot = feature_column.indicator_column(color_type)
demo(color_type_one_hot)

[[0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]]


In [17]:
#embedding
clarity = feature_column.categorical_column_with_vocabulary_list(
      'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=6)
demo(clarity_embedding)

[[-0.09706521 -0.47099394  0.19135481  0.09438153 -0.09205116  0.1184817 ]
 [-0.09706521 -0.47099394  0.19135481  0.09438153 -0.09205116  0.1184817 ]
 [-0.09706521 -0.47099394  0.19135481  0.09438153 -0.09205116  0.1184817 ]
 [ 0.13817176  0.0943175   0.4291614  -0.22348347  0.15736534 -0.17449962]
 [-0.02486611 -0.4617416   0.66259134  0.7038207   0.19487858 -0.44062388]
 [-0.09706521 -0.47099394  0.19135481  0.09438153 -0.09205116  0.1184817 ]
 [-0.42026365  0.11238766 -0.320006   -0.29430935  0.23472856 -0.36989063]
 [-0.42011532  0.3674518  -0.4036029  -0.16147804  0.04117285  0.16688627]
 [-0.09706521 -0.47099394  0.19135481  0.09438153 -0.09205116  0.1184817 ]
 [-0.42026365  0.11238766 -0.320006   -0.29430935  0.23472856 -0.36989063]]


In [18]:
#hashed feature
clarity_hashed = feature_column.categorical_column_with_hash_bucket(
      'clarity', hash_bucket_size=5)
demo(feature_column.indicator_column(clarity_hashed))

[[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [19]:
#cross feature
#data yang di cross harus berupa string, categorical, atau bucketized
crossed_feature = feature_column.crossed_column([carat_buckets, color_type],
                                                hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [20]:
#Pilih feature column mana yang akan digunakan
feature_columns = []

In [21]:
#embedding column
clarity = feature_column.categorical_column_with_vocabulary_list(
      'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=8)
feature_columns.append(clarity_embedding)

In [22]:
# numeric column
for header in ['carat', 'depth', 'x', 'y', 'z']:
  feature_columns.append(feature_column.numeric_column(header))

In [23]:
#membuat feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [24]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

# **Mengekspor Model**

In [25]:
saving_path = ("/content/mymodel/") #path penyimpanan model
tf.saved_model.save(model, saving_path)

INFO:tensorflow:Assets written to: /content/mymodel/assets


In [26]:
#create, compile, and train the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
 
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1673/1686 [============================>.] - ETA: 0s - loss: 0.1890 - accuracy: 0.9170WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'Unnamed: 0': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'carat': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=float64>, 'cut': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=string>, 'color': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=string>, 'clarity': <tf.Tensor 'ExpandDims_2:0' shape=(None, 1) dtype=string>, 'depth': <tf.Tensor 'ExpandDims_5:0' shape=(None, 1) dtype=float64>, 'table': <tf.Tensor 'ExpandDims_7:0' shape=(None, 1) dtype=float64>, 'price': <tf.Tensor 'ExpandDims_6:0' shape=(None, 1) dtype=int64>, 'x': <tf.Tensor 'ExpandDims_8:0' shape=(None, 1) dtype=float64>, 'y': <tf.Tensor 'ExpandDims_9:0' shape=(None, 1) dty

In [27]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1686/1686 [==============================] - 3s 2ms/step - loss: 0.1074 - accuracy: 0.9545
Accuracy 0.9545050263404846


# **Packaging Model**


In [28]:
import os
import tarfile
def tardir(path, tar_name):
    with tarfile.open(tar_name, "w:gz") as tar_handle:
        for root, dirs, files in os.walk(path):
            for file in files:
                tar_handle.add(os.path.join(root, file))
tardir('root/my_model.pb', 'my_model.tar.gz')